In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train

,~,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,...,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period,Death
0,0,72,1,1,0,1,0,0,0,0,...,142.0,68.2,87,102,99,1,0,0,157,0
1,1,66,1,0,0,0,0,1,0,0,...,179.4,61.1,54,112,99,0,1,0,567,0
2,2,75,1,1,0,1,0,0,0,0,...,169.8,73.8,79,73,99,0,1,0,253,0
3,3,77,1,0,1,0,0,1,0,0,...,158.5,79.1,78,137,1,0,0,0,266,0
4,4,85,1,0,0,0,0,0,0,0,...,172.3,44.3,48,85,99,1,0,1,470,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,69,1,0,0,0,0,0,0,1,...,161.5,70.4,70,105,0,0,1,0,522,1
9996,9996,37,0,0,0,0,0,0,1,0,...,165.7,85.0,79,129,0,1,1,0,107,0
9997,9997,31,1,0,0,0,0,0,1,0,...,151.5,61.3,69,123,99,0,0,1,66,0
9998,9998,85,0,0,0,0,0,0,0,0,...,159.5,54.3,84,68,1,1,1,0,156,0


In [7]:
test

,No,AGE,Adenocarcinoma,Large cell carcinoma,Squamous cell carcinoma,TX,T0,T1,T1a,T1b,...,Smoke,Height,Weight,FEV1_FVC_P,DLCO_VA_P,EGFR mutation Detection,Operation,Chemotherapy,Radiation Therapy,Survival period
0,0,35,1,0,0,0,0,0,0,0,...,1,174.2,69.1,82,43,0,0,1,0,355
1,1,60,1,0,0,0,0,0,0,1,...,0,158.8,67.4,52,86,0,0,0,1,166
2,2,45,1,1,1,1,0,0,0,0,...,0,159.7,71.2,49,68,99,1,1,1,623
3,3,31,0,0,0,1,0,0,0,0,...,1,153.6,71.0,82,87,1,1,0,0,445
4,4,55,0,0,1,0,0,0,1,0,...,1,163.6,72.0,68,127,99,0,1,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,47,1,1,0,0,0,0,0,0,...,0,147.6,79.0,40,79,99,1,0,0,104
4996,4996,44,1,0,1,0,0,1,0,0,...,2,151.0,41.6,69,67,99,0,0,0,42
4997,4997,80,0,0,1,0,0,0,1,0,...,0,157.8,58.2,40,137,0,1,1,0,190
4998,4998,69,1,0,0,0,0,0,0,0,...,2,167.5,56.7,89,77,0,1,1,1,635


In [19]:
train.isnull().sum()[train.isnull().sum()>0]


Series([], dtype: int64)

In [20]:
test.isnull().sum()[test.isnull().sum()>0]

Series([], dtype: int64)

In [21]:
train.columns

Index(['~', 'AGE', 'Adenocarcinoma', 'Large cell carcinoma',
       'Squamous cell carcinoma', 'TX', 'T0', 'T1', 'T1a', 'T1b', 'T1c', 'T2',
       'T2a', 'T2b', 'T3', 'T4', 'N1', 'N2', 'N3', 'M1a', 'M1b', 'M1c',
       'Type of Drink', 'Smoke', 'Height', 'Weight', 'FEV1_FVC_P', 'DLCO_VA_P',
       'EGFR mutation Detection', 'Operation', 'Chemotherapy',
       'Radiation Therapy', 'Survival period', 'Death'],
      dtype='object')

In [26]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [31]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.786


In [42]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

# XGBoost 모델 생성
model = xgb.XGBClassifier()

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, verbose=2)

# GridSearchCV로 최적의 하이퍼파라미터 찾기
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

preds = grid_search.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, preds)
print("Test Accuracy:", accuracy)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=1, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.7, gamma=0, learnin

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# GradientBoostingClassifier 모델 생성 및 훈련
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
preds = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, preds)
print("Accuracy:", accuracy)


Accuracy: 0.785
